<h2>文章を単語IDで表現する</h2>

<h3>事前加工したテキストファイルを読込みます</h3>

In [ ]:
import pickle
document_file_loaded = open("./document1.pkl","rb")
documents = pickle.load(document_file_loaded)
labels_file_loaded = open("./label1.pkl","rb")
labels = pickle.load(labels_file_loaded)

<h4>１番目のファイルの先頭部分を表示して内容を確認してみます</h4>

In [ ]:
documents[0][:200]

In [ ]:
print(labels[0])
print(labels[-1])

<h3>MeCab をインポートします</h3>

<h5>MeCab 関連の処理はこちらを参考にしました</h5>

- [キカガク：自然言語処理の基礎（PyTorch）](https://free.kikagaku.ai/tutorial/basic_of_nlp/learn/pytorch_text_classification)


In [ ]:
import MeCab

In [ ]:
mecab = MeCab.Tagger("-d /var/lib/mecab/dic/ipadic-utf8 -Ochasen")

<h4>テキストから名詞だけを抽出する</h4>

In [ ]:
mecab.parse('ブートキャンプする').split('\n')

<h4>'EOS' と ' ' を取り除き、タブを取り除きます。最後に '名詞' を含む行の先頭文字を取得します</h4>

In [ ]:
def list_nouns(sentense):
    nouns = []
    word_pieces = mecab.parse(sentense)
    word_pieces = word_pieces.split('\n')[:-2]
    for piece in word_pieces:
        item = piece.split('\t')
        if '名詞' in item[3]:        # <=== changed to 4 for default output format
            nouns.append(item[0])
    return nouns

In [ ]:
pieces = []
for document in documents:
    nouns = list_nouns(document)
    pieces.append(nouns)

<h4>先頭文書の名詞を確認します</h4>

- 文書は全部で 7367 あります

In [ ]:
print(pieces[0][:10])
print('ドキュメント数 ' + str(len(pieces)))

<h4>名詞の辞書を作成します</h4>

In [ ]:
from gensim import corpora, matutils

In [ ]:
dictionary_0 = corpora.Dictionary(pieces)
print('名詞の数 ' + str(len(dictionary_0)))

<h4>出現回数が 20以下の名詞を削除</h4>

In [ ]:
dictionary_0.filter_extremes(no_below=20)

In [ ]:
print('名詞の数 ' + str(len(dictionary_0)))

<h4>単語と ID の関係を確認します</h4>

In [ ]:
dictionary_0.token2id

<h4>文書に含まれる名詞全体のベクトル表現を作ってみます（各要素は名詞の発生件数で、名詞が発生した順序は考慮されません）</h4>

- 今回は利用しませんが知ってると便利です

In [ ]:
num_pieces = len(dictionary_0)
x_token = []
for piece in pieces:
    bow = dictionary_0.doc2bow(piece)
    vec = matutils.corpus2dense([bow], num_pieces).T[0]
    x_token.append(vec)
x_token[0]

<h3>パディング用の意味のない単語を追加してマージします（'pad'という単語を 0 にアサインします）</h3>

In [ ]:
pad_corpus = [['pad']]   # for pading token
dictionary = corpora.Dictionary(pad_corpus)
dictionary.token2id

In [ ]:
dictionary.merge_with(dictionary_0)
dictionary.token2id

<h3>文書（名詞）を辞書の名詞 ID で置換します</h3>
<h4>辞書にない単語は無視します</h4>

- -1 は辞書にない単語（出現回数が 20 以下）

In [ ]:
x_idx = []
for piece in pieces:
    idx_0 = dictionary.doc2idx(piece)
    idx_1 = [a for a in idx_0 if a  != -1]
    x_idx.append(idx_1)

##### 最初の文書（名詞）の長さと、辞書の名詞 ID で置換された内容を確認します

In [ ]:
print(len(x_idx[0]))
print(x_idx[0])

<h4>もっとも長いリストを確認します</h4>

In [ ]:
l = [len(a) for a in x_idx]
max_len = max(l)
max_len_idx =l.index(max_len)
print('max length', max_len)
print('max index', max_len_idx)

<h4>最も長い文書に長さを統一します</h4>

- その際、パディング文字 id 0 を使います

In [ ]:
x_pad = []
for idx in x_idx:
    for a in range(max_len - len(idx)):
        idx.append(0)
    x_pad.append(idx)

<h4>先頭の文書で確認します</h4>

In [ ]:
print(len(x_pad[0]))
print(x_pad[0][:200])

<h4>今回はメモリ節約のために長さを切詰めます</h4>

In [ ]:
sentence_size = 200
x_pad2 = []
for idx in x_idx:
    n = len(idx)
    if n > sentence_size:        
        for a in range(0, n - sentence_size ):
            idx.pop()
    else:
        for a in range(sentence_size - n):
            idx.append(0)  # idx.insert(0, 0)
    x_pad2.append(idx)

In [ ]:
print(len(x_pad2[0]))
print(x_pad2[0])

<h4>長さを確認します</h4>

In [ ]:
l_pad2 = [len(a) for a in x_pad2]
print('max length', max(l_pad2))
print('min length', min(l_pad2))

<h3>名詞の数を確認します</h3>

- この vocab_size はモデルトレーニングで指定する値になります</h4>


In [ ]:
vocab_size = len(dictionary)
print(vocab_size)

<h4>ここまでのデータを一旦エクスポートします</h4>

In [ ]:
import numpy as np
x = np.array(x_pad2)
y = np.array(labels)

In [ ]:
np.savez('docdata1', x, y)